# Libraries & Configuration

In [38]:
"""
By default the below script will download most recent 100 comments of a youtube video
You can change the default filter (line 30 onwards)

Variables :
COMMENT_LIMIT : How many comments you want to download 
SORT_BY_POPULAR : filter comments by popularity (0 for True , 1 for false)
SORT_BY_RECENT : filter comments by recently posted (0 for True , 1 for false)
"""

import pandas as pd
import json
import os
import sys
import re
import time

import requests

# pandas dataframe display configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_service_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
# csv file name
FILE_NAME = 'ytb_comments.csv'

# set parameters
# filter comments by popularity or recent, 0:False, 1:True
SORT_BY_POPULAR = 0
# default recent False, change to 1 to download latest comments
SORT_BY_RECENT = 0
# set comment limit
COMMENT_LIMIT = 100000

YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

## Utility Function

In [201]:
def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default


def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']
    
    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token']}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)

def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)

    html = response.text
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:
        return # Unable to extract configuration
    if language:
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language

    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))

    section = next(search_dict(data, 'itemSectionRenderer'), None)
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:
        # Comments disabled?
        return

    needs_sorting = sort_by != SORT_BY_POPULAR
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)

        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        if needs_sorting:
            sort_menu = next(search_dict(response, 'sortFilterSubMenuRenderer'), {}).get('subMenuItems', [])
            if sort_by < len(sort_menu):
                continuations = [sort_menu[sort_by]['serviceEndpoint']]
                needs_sorting = False
                continue
            raise RuntimeError('Failed to set sorting')

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    # Process continuations for comments and replies.
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # Process the 'Show more replies' button
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])

        for comment in reversed(list(search_dict(response, 'commentRenderer'))):
            yield {'cid': comment['commentId'],
                   'text': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]),
                   'time': comment['publishedTimeText']['runs'][0]['text'],
                   'author': comment.get('authorText', {}).get('simpleText', ''),
                   'channel': comment['authorEndpoint']['browseEndpoint'].get('browseId', ''),
                   'votes': comment.get('voteCount', {}).get('simpleText', '0'),
                   'photo': comment['authorThumbnail']['thumbnails'][-1]['url'],
                   'heart': next(search_dict(comment, 'isHearted'), False)}

        time.sleep(sleep)

def search_dict(partial, search_key):
    stack = [partial]
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)

In [202]:
def main(url):
    """
    This function will save the comments into a dataframe and output a csv file
    By default, it will append the comments in csv, not overwriting them, change it in line 34
    To preview the comments in json, uncomment the lines - 22 and 23
    """
    df_comment = pd.DataFrame()
    try:
        youtube_url = url
        limit = COMMENT_LIMIT

        print('Downloading Youtube comments for video:', youtube_url)

        count = 0

        start_time = time.time()

        for comment in download_comments(youtube_url):

            df_comment = df_comment.append(comment, ignore_index=True)
            print(df_comment)

            # comments overview in json
            # comment_json = json.dumps(comment, ensure_ascii=False)
            # print(comment_json)

            count += 1

#             if limit and count >= limit:
#                 break

#         print("DataFrame Shape: ",df_comment.shape,"\nComment DataFrame: ")
#         display(df_comment)

#         if not os.path.isfile(FILE_NAME):
#             df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
#         else:  # else it exists so append without writing the header
#             df_comment.to_csv(FILE_NAME, mode='a', encoding='utf-8', index=False, header=False)

        print('\n[{:.2f} seconds] Done!'.format(time.time() - start_time))

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)

# Save Comments

## Single Video Link

In [203]:
# dumping youtube comments

""" 
Dump comments to a csv  from a single video

"""
# youtube_URL = 'https://youtu.be/A7yHrPoZ2mE'
youtube_URL = 'https://www.youtube.com/watch?v=VmEU22NYkEs'
main(youtube_URL)

Error: HTTPSConnectionPool(host='www.youtube.com', port=443): Max retries exceeded with url: /watch?v=VmEU22NYkEs (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000024A7A94A0D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


AssertionError: 

In [ ]:
df_comment = pd.read_csv('ytb_comments.csv')
df_comment.shape

## List of Links

In [ ]:
"""
Dump to a csv from a a list with video links
"""
ytb_video_list = ['https://youtu.be/0sOvCWFmrtA',
                  'https://www.youtube.com/watch?v=TuIgtitqJho',
                  'https://www.youtube.com/watch?v=hinZO--TEk4',
                  'https://youtu.be/q6EoRBvdVPQ?list=PLFsQleAWXsj_4yDeebiIADdH5FMayBiJo']

for video_link in ytb_video_list:
    main(video_link)

In [ ]:
df_comment = pd.read_csv('ytb_comments.csv')
df_comment.shape

## Links From DataFrame

In [ ]:
"""
Dump comments to a csv by parsing links from a csv with video links

Example -
Create a csv with one column titled 'link'
a sample is given below

'ytb_video_list.csv'

link
https://www.youtube.com/watch?v=-t_uhBBDbA4
https://www.youtube.com/watch?v=75vjjRza7IU
https://www.youtube.com/watch?v=j6dmaPzOBHY
https://www.youtube.com/watch?v=Yj2efyQV1RI
https://www.youtube.com/watch?v=HV652F7U6Qs
https://www.youtube.com/watch?v=47iXEucg3eo
https://www.youtube.com/watch?v=ofHXBLEE3TQ
https://www.youtube.com/watch?v=X6lGqSfVRT8
https://www.youtube.com/watch?v=a_-z9FhGBrE
https://www.youtube.com/watch?v=wTUM_4cVlE4


"""

youtube_data = pd.read_csv("YouTube_data.csv")
youtube_data.url[:3].map(lambda x: main(x))

## Preview Output CSV

In [ ]:
df_comment = pd.read_csv('./ytb_comments.csv')

print(f"{df_comment.shape[0]} rows, {df_comment.shape[1]} columns")
df_comment.head()

## Reference

1. [https://github.com/egbertbouman/youtube-comment-downloader](https://github.com/egbertbouman/youtube-comment-downloader)

In [ ]:
from selenium import webdriver
import time
import os
import csv
import pandas as pd
from math import ceil
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import time

In [ ]:
comment_list = []
comment_reply = []
def ScrapComment(url):
    option = webdriver.FirefoxOptions()
    option.add_argument("--headless")
    driver = webdriver.Chrome(executable_path=GeckoDriverManager().install(), options=option)
    driver.get(url)
    prev_h = 0
    while True:
        height = driver.execute_script("""
                function getActualHeight() {
                    return Math.max(
                        Math.max(document.body.scrollHeight, document.documentElement.scrollHeight),
                        Math.max(document.body.offsetHeight, document.documentElement.offsetHeight),
                        Math.max(document.body.clientHeight, document.documentElement.clientHeight)
                    );
                }
                return getActualHeight();
            """)
        driver.execute_script(f"window.scrollTo({prev_h},{prev_h + 200})")
        # fix the time sleep value according to your network connection
        time.sleep(1)
        prev_h +=200  
        if prev_h >= height:
            break
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    title_text_div = soup.select_one('#container h1')
    title = title_text_div and title_text_div.text
    comment_div = soup.select("#content #content-text")
    comment_r = soup.select('style-scope yt-formatted-string')
    global comment_list
    global comment_reply
    comment_list = [x.text for x in comment_div]
    comment_reply = [x for x in comment_div]
    print(f'comment_list {comment_list}')
    print(f'title {title}')


if __name__ == "__main__":

    urls = [
        "https://www.youtube.com/watch?v=VmEU22NYkEs"
    ]
    ScrapComment(urls[0])

In [ ]:
len(comment_list)

In [ ]:
new_comment_list = []
for i in comment_list:
    if i != '':
        new_comment_list.append(i)

In [ ]:
df = pd.DataFrame(new_comment_list,columns=['comments'])

In [ ]:
df.to_csv('ytb_comments.csv',index=False)

In [ ]:
# df['comments'] = comment_list

In [206]:
df.head()

,comments
0,"2.5 million views, say MashALLAH :)"
1,Allah is medicine of every pain
2,"Very peaceful song....love from india,Allah!!!..."
3,I am hindu but this song gives peace to my heart
4,I am facing a problem!I can not sleep without ...
...,...
1759,Mai Mohammad Akbar hu Hyderabad India se aor ...
1760,roshan kare rasta tera
1761,ALLAH
1762,ALLAH


In [ ]:
new_comment_list

In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
# from google_trans_new import google_translator
# translator = google_translator()
translate_comments = []
# for i in new_comment_list:
#     translate_text = translator.translate('this is',lang_tgt='en') 
#     translate_comments.append(translate_text)
# translate_comments
# translate_text = translator.translate('this is',lang_tgt='en') 
# translate_text

In [ ]:
text_clean = []
for i in new_comment_list:
    text = re.sub(r"(@\(\w+:\/\/\S+)", "", i)
#     text_clean.append(re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", i))
    text_clean.append(re.sub(r"\n",'',i))
text_clean

In [ ]:
df = pd.DataFrame(text_clean,columns=['comments'])

In [44]:
# df.to_csv('ytb_comments.csv',index=False)
import pandas as pd
df = pd.read_csv('ytb_comments.csv')

In [45]:
df.isnull().sum()

comments    0
dtype: int64

In [46]:
def entityRecognition(doc):
   if doc.ents:
     for ent in doc.ents:
        print(f"{ent.text} - {ent.label_} - {spacy.explain(ent.label_)}")
     else:
        print('No Entity')

In [47]:
doc = nlp(u'{i}')
entityRecognition(doc)

In [55]:
from googletrans import Translator
translator = Translator()

trans_comments = []
for i in df.comments:
    trans = translator.translate(i,dest="en")
    trans_comments.append(trans.text)
    print(trans.text)

2.5 million views, say MashALLAH :)
Allah is medicine of every pain
Very peaceful song....Love from India,Allah!!!Keep all my Muslim brothers safe....
I am hindu but this song gives peace to my heart
I am facing a problem!I can not sleep without listening to these 3 soothing songs-1. Noor-e-ilahi2. Noor-e-azal3. Pardadari.️ Watching from BangladeshEdit- now added 'faryaad'
I m hindu but every time I listen this song it gives me a heavenly peace my god I can't even explain in words ....my love for allah is limitless like a sky coz I know mera.khuda allah mera ishwar ye to ek hi hai sirf alag alag logon ne.naam.alag rkh diye ....love this song
Am not A Muslim but I Listen it everyday when I get time ️.I love It ️
Purest form of music..... I'm hindu but I can feel the holy presence of God everywhere...
I am hindu but love to hear this song....And Atif is just awesome
Today I was fasting Rosa and after iftar I started feeling very uneasy because of certain problems in my life and I started

I am listening this looping again and again I can't stop hearing this beautiful voice and lyrics !!!
Its a masterpiece ️ goosebumps ️
Soul touching masterpiece ️
Mashallah they both voice are touching my heart
Mr really write this voice Mashallah and I am listening now Ya Hamd..., Noor e Azal
May you live long for singing in such a melodious voice for us.
Love this ️️️️️
Magical voice of atif aslam
️️️ BESHAK ALLAH HU AKBAR ️️️ SALLALLAHU ALAIHI WA AALIHI WA ASHABIHI WASALLAM ️️️
Both are legends.no matchLove from India
This is the memory of Ramadan
really Nice Voice Atif Aslam and Abida Parveen
️️🇮🇳🇮🇳Whoever is reading it May Allah bless you
Awesome. Just put your headphones and you are in heaven. Abida Parveen is a legend but in this hamad Atif Aslam set a new benchmark for himself. God bless you both.
Very beautiful naat
Today I m hearing this song again and again God ,help me I m totally broken
Subhanallah ️ mashallah ️
Listen this 100+ time. Soulful️
Aye khuda... jau kaha gumshuda

Am. Here on Eid I found this on this Ramadaan. Hamdulilah Ramadaan is complete and today is Eid in India. But we can see the current situation in Palestine. Brothers and sisters don't forget to pray for our Palestinian people In your Salah's and prayers. Jazak Allah khair
Masha Allah what a melodious voice of abida ji and atif aslam sir love from India
Whenever I think of doing a sin I come here :)
So peaceful
Proud To be A MuslimProud to be an Ummati of a Hazrat Muhammad (S.A.W) Ya Allah, any Muslim's prayer is worshiped or worshiped, Ameen Suma Ameen
Who is still listening in 2020 Very peaceful voice and words
️️️️ giving peace to my soul
GOSSEEBUMPS
Such kalams deserved more views
In this Ramadan I can't hear songs & I miss you Atif.....But, Atif has some good number of ghazal collection......So, I'm here.....
Heard it two years ago in Ramadan. Listening again today, I don't know the time
Masha'Allah lovely sweet naats 🇵🇰🇵🇰🇹🇷🇹🇷
Love it  i started crying yar
I don't know but I always

ReadTimeout: The read operation timed out

In [61]:
trans = translator.translate('Very heart broken naat and baishak Allah is me...',dest='ur')
trans.text

'اللہ ہر درد کی دوا ہے۔'

In [56]:
len(trans_comments)

335

In [17]:
# trans_comments_n = []
# for i in trans_comments:
#     trans_comments_n.append(i)
# len(trans_comments_n)

530

In [20]:
trans_comments

['2.5 million views, say MashALLAH :)',
 'Allah is medicine of every pain',
 'Very peaceful song....Love from India,Allah!!!Keep all my Muslim brothers safe....',
 'I am hindu but this song gives peace to my heart',
 "I am facing a problem!I can not sleep without listening to these 3 soothing songs-1. Noor-e-ilahi2. Noor-e-azal3. Pardadari.️ Watching from BangladeshEdit- now added 'faryaad'",
 "I m hindu but every time I listen this song it gives me a heavenly peace my god I can't even explain in words ....my love for allah is limitless like a sky coz I know mera.khuda allah mera ishwar ye to ek hi hai sirf alag alag logon ne.naam.alag rkh diye ....love this song",
 'Am not A Muslim but I Listen it everyday when I get time ️.I love It ️',
 "Purest form of music..... I'm hindu but I can feel the holy presence of God everywhere...",
 'I am hindu but love to hear this song....And Atif is just awesome',
 "Today I was fasting Rosa and after iftar I started feeling very uneasy because of cer

In [57]:
df = pd.DataFrame(trans_comments,columns=['trans_comments'])
df.to_csv('ytb_trans_comments.csv',index=False)

In [58]:
df

,trans_comments
0,"2.5 million views, say MashALLAH :)"
1,Allah is medicine of every pain
2,"Very peaceful song....Love from India,Allah!!!..."
3,I am hindu but this song gives peace to my heart
4,I am facing a problem!I can not sleep without ...
5,I m hindu but every time I listen this song it...
6,Am not A Muslim but I Listen it everyday when ...
7,Purest form of music..... I'm hindu but I can ...
8,I am hindu but love to hear this song....And A...
9,Today I was fasting Rosa and after iftar I sta...


In [39]:
len(df.trans_comments)

140

In [30]:
def entityRecognition(doc):
   if doc.ents:
     for ent in doc.ents:
        print(f"{ent.text} - {ent.label_} - {spacy.explain(ent.label_)}",end='\n')
     else:
        print('No Entity')

In [31]:
for i in df.trans_comments:
    doc = nlp(i)
    entityRecognition(doc)

2.5 million - CARDINAL - Numerals that do not fall under another type
MashALLAH - CARDINAL - Numerals that do not fall under another type
No Entity
India - GPE - Countries, cities, states
Muslim - NORP - Nationalities or religious or political groups
No Entity
3 - CARDINAL - Numerals that do not fall under another type
Watching - PRODUCT - Objects, vehicles, foods, etc. (not services)
No Entity
hai - GPE - Countries, cities, states
logon ne.naam.alag - PERSON - People, including fictional
rkh diye - PERSON - People, including fictional
No Entity
Muslim - NORP - Nationalities or religious or political groups
No Entity
Atif - PERSON - People, including fictional
No Entity
Today - DATE - Absolute or relative dates or periods
Rosa - PERSON - People, including fictional
No Entity
Muslim - NORP - Nationalities or religious or political groups
No Entity
Humans - ORG - Companies, agencies, institutions, etc.
No Entity
God - PERSON - People, including fictional
muslim - NORP - Nationalities or 

Atif Saeed " - PERSON - People, including fictional
Banda Mushkil - PERSON - People, including fictional
No Entity
Jahaan Bhi Main Dekhun - PERSON - People, including fictional
Bada Hai - PERSON - People, including fictional
 Hamesha - PERSON - People, including fictional
Gumshuda Mera Jahaan - PERSON - People, including fictional
Mera - PERSON - People, including fictional
Tera - ORG - Companies, agencies, institutions, etc.
Yahi - ORG - Companies, agencies, institutions, etc.
Light - PRODUCT - Objects, vehicles, foods, etc. (not services)
Maula mann - PERSON - People, including fictional
Maula mann ko - PERSON - People, including fictional
Maula mann - PERSON - People, including fictional
Maula mann ko - PERSON - People, including fictional
Jo hai kainaat - PERSON - People, including fictional
mein har - PERSON - People, including fictional
hai - PERSON - People, including fictional
 Tera - PERSON - People, including fictional
Tera - ORG - Companies, agencies, institutions, etc.
Tera

In [32]:
def showLemmas(text):
    var = nlp(text)
    for token in var:
        print(f'{token.text:{12}} {token.pos_:{6}} {token.lemma:<{22}} {token.lemma_}')

In [37]:
for i in df.trans_comments:
    showLemmas(i)

2.5          NUM    151696752626035455     2.5
million      NUM    17365054503653917826   million
views        NOUN   2608925738682777778    view
,            PUNCT  2593208677638477497    ,
say          VERB   8685289367999165211    say
MashALLAH    X      1408768325840884581    mashallah
:)           SPACE  5920004935509210957    :)
Allah        PROPN  14001085240282908743   Allah
is           AUX    10382539506755952630   be
medicine     NOUN   322027408632210549     medicine
of           ADP    886050111519832510     of
every        DET    2739543297294151269    every
pain         NOUN   8558038558878807367    pain
Very         ADV    9548244504980166557    very
peaceful     ADJ    12583384013854733425   peaceful
song         NOUN   9688994450605453460    song
....         PUNCT  8679283378860202508    ....
Love         NOUN   3702023516439754181    love
from         ADP    7831658034963690409    from
India        PROPN  13756031414435721121   India
,            PUNCT  259320867763

as           SCONJ  7437575085468336610    as
I            PRON   4690420944186131903    I
was          AUX    10382539506755952630   be
feeling      VERB   5741770584995928333    feel
really       ADV    14653600329166397554   really
bad          ADJ    12342627399458421040   bad
in           ADP    3002984154512732771    in
my           PRON   227504873216781231     my
heart        NOUN   12419898511841729938   heart
and          CCONJ  2283656566040971221    and
I            PRON   4690420944186131903    I
cried        VERB   5286809987425472328    cry
to           ADP    3791531372978436496    to
Allah        PROPN  14001085240282908743   Allah
I            PRON   4690420944186131903    I
just         ADV    7148522813498185515    just
called       VERB   14229572451745258962   call
upon         SCONJ  12776617025319584140   upon
Allah        PROPN  14001085240282908743   Allah
and          CCONJ  2283656566040971221    and
all          PRON   13409319323822384369   all
of         

Love         VERB   3702023516439754181    love
from         ADP    7831658034963690409    from
india        PROPN  9504827836828505330    india
...          PUNCT  10875615029400813363   ...
             SPACE  8532415787641010193     
divided      VERB   9657931866832838961    divide
by           ADP    16764210730586636600   by
border       NOUN   17575469328687188493   border
United       PROPN  13226800834791099135   United
by           ADP    16764210730586636600   by
soul         NOUN   10083117423477906790   soul
..           PUNCT  8848021949395737739    ..
just         ADV    7148522813498185515    just
superb       ADJ    762349577065204215     superb
...          PUNCT  10875615029400813363   ...
head         NOUN   8419699711262724776    head
falls        VERB   2049701496445196583    fall
in           ADP    3002984154512732771    in
gratitude    NOUN   3982778831050600591    gratitude
.            PUNCT  12646065887601541794   .
Allah        PROPN  14001085240282908743  

is           AUX    10382539506755952630   be
play         VERB   8228585124152053988    play
important    ADJ    5951875591235763184    important
role         NOUN   15289190992820506615   role
to           PART   3791531372978436496    to
spread       VERB   1686152860662068943    spread
islam        PROPN  9775269960725595335    islam
in           ADP    3002984154512732771    in
whole        ADJ    16948554243429412012   whole
world        NOUN   1703489418272052182    world
ones         NOUN   17454115351911680600   one
again        ADV    4502205900248518970    again
..           PUNCT  8848021949395737739    ..
love         VERB   3702023516439754181    love
you          PRON   7624161793554793053    you
atif         VERB   1312366571600476618    atif
your         PRON   1572612192562026184    your
legend       NOUN   17067105940613322182   legend
Masha        PROPN  11304513591069003651   Masha
Allah        PROPN  14001085240282908743   Allah
!            PUNCT  174948030463125

Very         ADV    9548244504980166557    very
peaceful     ADJ    12583384013854733425   peaceful
song         NOUN   9688994450605453460    song
Two          NUM    11711838292424000352   two
legends      NOUN   17067105940613322182   legend
of           ADP    886050111519832510     of
my           PRON   227504873216781231     my
Country      NOUN   12290671265767728302   country
Its          PRON   12513610393978129441   its
on           ADP    5640369432778651323    on
repeat       NOUN   11789760439429990770   repeat
in           ADP    3002984154512732771    in
house        NOUN   9471806766518506264    house
,            PUNCT  2593208677638477497    ,
since        SCONJ  10066841407251338481   since
last         ADJ    10321518907502812892   last
2            NUM    15180167692696242062   2
days         NOUN   1608482186128794349    day
..           PUNCT  8848021949395737739    ..
so           ADV    9781598966686434415    so
soothing     ADJ    591144222052831411     sooth

...

precious     ADJ    5546633287517827381    precious
gems         NOUN   9287986770314362048    gem
Pakistani    ADJ    14429195809366666767   pakistani
music        NOUN   13617878141848112644   music
industry     NOUN   16409696763006672153   industry
_            PUNCT  456                    _
️            PUNCT  10886762227117140680   ️
_            PUNCT  456                    _
!            PUNCT  17494803046312582752   !
!            PUNCT  17494803046312582752   !
             SPACE  8532415787641010193     
Loved        VERB   3702023516439754181    love
it           PRON   10239237003504588839   it
🇮            PROPN  5057829697812157309    🇮
🇳            VERB   16331524874405447071   🇳
So           ADV    9781598966686434415    so
impressive   ADJ    13718441613973299639   impressive
that         SCONJ  4380130941430378203    that
             SPACE  8532415787641010193     
I            PRON   4690420944186131903    I
can          AUX    6635067063807956629    can
listen  

am           AUX    10382539506755952630   be
hindu.but    NOUN   13389125321194477497   hindu.but
i            PRON   4690420944186131903    I
feel         VERB   5741770584995928333    feel
in           ADP    3002984154512732771    in
masa         PROPN  6135777023270064070    masa
allah        PROPN  3018971744045221257    allah
.            PUNCT  12646065887601541794   .
Allah        PROPN  14001085240282908743   Allah
Pak          PROPN  12805948618854367171   Pak
gave         VERB   11640825575873464194   give
me           PRON   4690420944186131903    I
hope         NOUN   4429974322456332988    hope
through      ADP    18216413589307435838   through
this         DET    1995909169258310477    this
kalam        NOUN   4446636690918956005    kalam
in           ADP    3002984154512732771    in
your         PRON   1572612192562026184    your
voice        NOUN   257396040179354343     voice
at           ADP    11667289587015813222   at
the          DET    7425985699627899538    the

Its          PRON   12513610393978129441   its
a            DET    11901859001352538922   a
masterpiece  NOUN   6410384827905653140    masterpiece
️            NUM    10886762227117140680   ️
goosebumps   NOUN   4853141342160931285    goosebump
️            SPACE  10886762227117140680   ️
Soul         NOUN   10083117423477906790   soul
touching     ADJ    12291118906283350175   touching
masterpiece  NOUN   6410384827905653140    masterpiece
️            SPACE  10886762227117140680   ️
Mashallah    PROPN  5446068900232834748    Mashallah
they         PRON   16875582379069451158   they
both         DET    7111508780595485950    both
voice        NOUN   257396040179354343     voice
are          AUX    10382539506755952630   be
touching     VERB   14181128892556480885   touch
my           PRON   227504873216781231     my
heart        NOUN   12419898511841729938   heart
Mr           PROPN  17044766764529665155   Mr
really       ADV    14653600329166397554   really
write        VERB   155974

Allah        PROPN  14001085240282908743   Allah
is           AUX    10382539506755952630   be
enough       ADJ    5083403373732563023    enough
for          ADP    16037325823156266367   for
me           PRON   4690420944186131903    I
.            PUNCT  12646065887601541794   .
When         SCONJ  15807309897752499399   when
Atif         PROPN  8094378636411740158    Atif
Saeed        PROPN  10710228592232616889   Saeed
"            PUNCT  15884554869126768810   "
Meri         PROPN  8012618251107674303    Meri
Mola         PROPN  13251442547095276222   Mola
Saath        PROPN  5450411704035079880    Saath
Day          PROPN  13415491272735578485   Day
Mara         PROPN  18259313333564410587   Mara
Haath        PROPN  9199184629397778953    Haath
Tham         PROPN  15847925472412877076   Tham
Le           PROPN  17405251050291145966   Le
Tera         PROPN  356821824845976211     Tera
Banda        PROPN  999982642429369561     Banda
Mushkil      PROPN  5414917960950288359    Mushk

Chali        PROPN  17939402068656998386   Chali
Mauj         PROPN  5664456670518465831    Mauj
Noor         PROPN  2467458250763973546    Noor
Ki           PROPN  8322114704296378710    Ki
            SPACE  14481569651509769704   
Huyi         PROPN  4537677112467938028    Huyi
Puri         PROPN  9840639716576025643    Puri
Roshni       PROPN  7243675708265243268    Roshni
Ki           PROPN  8322114704296378710    Ki
Har          PROPN  14407892794660518610   Har
Dua          PROPN  2919697881244208720    Dua
            SPACE  10975671256698354229   
Noor         PROPN  2467458250763973546    Noor
-            PUNCT  9153284864653046197    -
e            NOUN   1720370409040345145    e
-            PROPN  9153284864653046197    -
Azal         PROPN  9382665524146677435    Azal
Noor         PROPN  2467458250763973546    Noor
-            PUNCT  9153284864653046197    -
e            NOUN   1720370409040345145    e
-            NOUN   9153284864653046197    -
Khuda        PROPN  159